## Imports

In [2]:
pip install sagemaker

  Stored in directory: C:\Users\LAksh\AppData\Local\pip\Cache\wheels\68\dc\99\76b26832ce6dc6a5c1e9c343f262662958f48c14047701c0b4
  Stored in directory: C:\Users\LAksh\AppData\Local\pip\Cache\wheels\37\42\d8\1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31
Successfully built sagemaker protobuf3-to-dict
  Found existing installation: zipp 0.3.3
    Uninstalling zipp-0.3.3:
      Successfully uninstalled zipp-0.3.3
  Found existing installation: importlib-metadata 0.0.0
    Uninstalling importlib-metadata-0.0.0:
      Successfully uninstalled importlib-metadata-0.0.0
  Found existing installation: packaging 19.0
    Uninstalling packaging-19.0:
      Successfully uninstalled packaging-19.0
Note: you may need to restart the kernel to use updated packages.


fastai 1.0.59 requires nvidia-ml-py3, which is not installed.


In [7]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

## Create Model

Now we use the License Plate Detection & Recognition Model Package to create a model.

In [3]:
from src.model_package_arns import ModelPackageArnProvider

In [4]:
from sagemaker import ModelPackage
modelpackage_arn = ModelPackageArnProvider.get_model_package_arn(sess.boto_region_name)

# Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                                      model_package_arn = modelpackage_arn,
                                      sagemaker_session = sess,
                                      predictor_cls = predict_wrapper)
# ARN
print(modelpackage_arn)

arn:aws:sagemaker:us-east-2:057799348421:model-package/license-plate-detector15714142-f5bfb4d51b9e4fcfa2aa360e8a3a88f9


### Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference

In [5]:
# Deploy the model
predictor = model.deploy(1, 'ml.m4.xlarge', endpoint_name='license-plate-inference')

--------------------------------------------------------------------------------------!

### Testing on a single file 

The model takes an image file (.jpg/.png) as input. We give the location of the sample input file and prepare it for payload. An example has been shown in the following code shell.


In [6]:
file_name = 'data/test_files/test_image.jpeg'

In [7]:
endpoint_name='license-plate-inference'

#### Preparing the input file for payload and viewing the response

In [8]:
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

# View the prediction
print(result)

KL40L5577


## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

### Batch Transform Input Preparation


In [119]:
common_prefix = "license_plate_recognition"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
transform_input_folder = "data/transform"
transform_input = sess.upload_data(transform_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-298841451579/license_plate_recognition/batch-inference-input-data


### Run Batch Transform

Now that our batch transform input is setup, we run the transformation job next

In [120]:
transform_output_folder = "license_plate_recognition/license_plate_output"
output_path = "s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = model.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               accept='image/jpeg')

transformer.transform(transform_input, content_type='image/jpeg')
transformer.wait()
print("Batch Transform output saved to " + transformer.output_path)

Using already existing model: license-plate-detector15714142-f5bfb4d5-2019-11-01-09-26-21-372


...................Starting the inference server with 4 workers.
[2019-11-01 11:50:18 +0000] [11] [INFO] Starting gunicorn 19.9.0
[2019-11-01 11:50:18 +0000] [11] [INFO] Listening at: unix:/tmp/gunicorn.sock (11)
[2019-11-01 11:50:18 +0000] [11] [INFO] Using worker: gevent
[2019-11-01 11:50:18 +0000] [15] [INFO] Booting worker with pid: 15
[2019-11-01 11:50:18 +0000] [16] [INFO] Booting worker with pid: 16
[2019-11-01 11:50:18 +0000] [17] [INFO] Booting worker with pid: 17
[2019-11-01 11:50:18 +0000] [18] [INFO] Booting worker with pid: 18
Using TensorFlow backend.
Using TensorFlow backend.
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: Fu


169.254.255.130 - - [01/Nov/2019:11:50:50 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [01/Nov/2019:11:50:50 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Endpint invoked
KL40L5577
169.254.255.130 - - [01/Nov/2019:11:50:51 +0000] "POST /invocations HTTP/1.1" 200 9 "-" "Go-http-client/1.1"
Endpint invoked
MY02ZR0
169.254.255.130 - - [01/Nov/2019:11:50:52 +0000] "POST /invocations HTTP/1.1" 200 7 "-" "Go-http-client/1.1"
Endpint invoked
J44NX0
169.254.255.130 - - [01/Nov/2019:11:50:53 +0000] "POST /invocations HTTP/1.1" 200 6 "-" "Go-http-client/1.1"
Endpint invoked
2019-11-01T11:50:50.080:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
KA02HJ4606
169.254.255.130 - - [01/Nov/2019:11:50:54 +0000] "POST /invocations HTTP/1.1" 200 10 "-" "Go-http-client/1.1"
Endpint invoked
KA02HJ4606
169.254.255.130 - - [01/Nov/2019:11:50:54 +0000] "POST /invocations HTTP/1.1" 200 10 "-" "Go-http-clien

In [121]:
# Print the outputs

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())
file_no=1
filenames=[]
for object_summary in my_bucket.objects.filter(Prefix="license_plate_recognition/license_plate_output/"):
   file_name = object_summary.key.split('/')[2]
   final_name='batch_results'+str(file_no)
   my_bucket.download_file("license_plate_recognition/license_plate_output/"+ file_name, final_name)
   filenames.append(final_name) 
   file_no+=1

In [122]:
#Creates a file "number_plates" with the recognized plate numbers

import os
with open('number_plates', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            inf=infile.read()
            inf=inf+"\n"
            outfile.write(inf)
            os.remove(fname)
plates = open('number_plates','r').read().splitlines()
print(plates)

['KL40L5577', 'TN38BR3036', 'MY02ZR0', 'J44NX0', 'KA02HJ4606']


In [124]:
# Clean Up Endpoint

predictor.delete_endpoint()